# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [45]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [46]:
file='output_data/cities.csv'
cities_df= pd.read_csv(file)
cities_df=cities_df[['City','Lat','Long','Max Temp','Humidity','Cloudiness','Wind Speed','Country','Date']]
cities_df.head(3)

,City,Lat,Long,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,vaini,-21.2000,-175.2000,75.36,88,75,10.36,TO,1661830666
1,xuanhua,40.6103,115.0447,75.65,46,0,5.39,CN,1661830666
2,plainview,40.7765,-73.4673,76.96,82,100,11.50,US,1661830667


In [47]:
cities_df.Humidity.max()

100

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [48]:
locations=cities_df[['Lat','Long']]
humidity=cities_df['Humidity']
fig = gmaps.figure()
fig.add_layer(gmaps.heatmap_layer(locations, weights=humidity,max_intensity=100,dissipating=True, point_radius=10))
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [49]:
cities_df.head(3)

,City,Lat,Long,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,vaini,-21.2000,-175.2000,75.36,88,75,10.36,TO,1661830666
1,xuanhua,40.6103,115.0447,75.65,46,0,5.39,CN,1661830666
2,plainview,40.7765,-73.4673,76.96,82,100,11.50,US,1661830667


In [50]:
reduced_cities_df= cities_df.loc[(cities_df['Max Temp']>75) & (cities_df['Max Temp']<=85) &
             (cities_df['Cloudiness']<10) & (cities_df['Wind Speed']<10),:]
reduced_cities_df
toptenresults=reduced_cities_df.iloc[:11,:]
toptenresults

,City,Lat,Long,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
1,xuanhua,40.6103,115.0447,75.65,46,0,5.39,CN,1661830666
42,albany,42.6001,-73.9662,79.21,75,0,5.93,US,1661830301
109,laguna,38.4210,-121.4238,84.40,51,0,9.22,US,1661830701
162,aksu,41.1231,80.2644,75.33,34,0,0.54,CN,1661830719
191,bethel,41.3712,-73.4140,75.65,88,0,6.91,US,1661830727
206,komsomolskiy,40.4272,71.7189,77.72,25,1,0.00,UZ,1661830732
209,west wendover,40.7391,-114.0733,83.41,18,0,4.61,US,1661830733
210,kashary,49.0400,41.0056,75.02,35,0,2.95,RU,1661830733
234,emba,48.8264,58.1442,80.13,25,8,2.13,KZ,1661830741
244,vicksburg,32.3526,-90.8779,80.28,97,0,0.00,US,1661830657


In [51]:
fig2 = gmaps.figure()
markers= gmaps.marker_layer(toptenresults[['Lat','Long']])
#add layer
fig2.add_layer(markers)
fig2

Figure(layout=FigureLayout(height='420px'))

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [52]:
toptenresults['Hotel Name']=' '
toptenresults.head()

C:\Users\ym881\AppData\Local\Temp\ipykernel_12012\519513691.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  toptenresults['Hotel Name']=' '


,City,Lat,Long,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
1,xuanhua,40.6103,115.0447,75.65,46,0,5.39,CN,1661830666,
42,albany,42.6001,-73.9662,79.21,75,0,5.93,US,1661830301,
109,laguna,38.4210,-121.4238,84.40,51,0,9.22,US,1661830701,
162,aksu,41.1231,80.2644,75.33,34,0,0.54,CN,1661830719,
191,bethel,41.3712,-73.4140,75.65,88,0,6.91,US,1661830727,


In [53]:
base_url= 'https://maps.googleapis.com/maps/api/place/nearbysearch/json?'
for index, row in toptenresults.iterrows():
    try:
        target_coordinates = f"{row['Lat']}, {row['Long']}"
        target_radius=5000
        target_search='hotel'
        params={'location': target_coordinates,
                'radius':target_radius,
                'keyword':target_search,
                'key':g_key}
        place_data=requests.get(base_url, params=params).json()
        toptenresults.loc[index, 'Hotel Name']=place_data['results'][0]['name']
    except:
        toptenresults.loc[index, 'Hotel Name']='none found'

In [54]:
hotel_df=toptenresults
hotel_df

,City,Lat,Long,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
1,xuanhua,40.6103,115.0447,75.65,46,0,5.39,CN,1661830666,Zhangjiakou Dream Home Luxiang Theme Hotel
42,albany,42.6001,-73.9662,79.21,75,0,5.93,US,1661830301,none found
109,laguna,38.4210,-121.4238,84.40,51,0,9.22,US,1661830701,Hampton Inn & Suites Sacramento-Elk Grove Lagu...
162,aksu,41.1231,80.2644,75.33,34,0,0.54,CN,1661830719,Hanting Hotels
191,bethel,41.3712,-73.4140,75.65,88,0,6.91,US,1661830727,Hampton Inn Danbury
206,komsomolskiy,40.4272,71.7189,77.72,25,1,0.00,UZ,1661830732,TANTANA HOTEL FERGANA
209,west wendover,40.7391,-114.0733,83.41,18,0,4.61,US,1661830733,Best Western Plus Wendover Inn
210,kashary,49.0400,41.0056,75.02,35,0,2.95,RU,1661830733,Гостиница N 1
234,emba,48.8264,58.1442,80.13,25,8,2.13,KZ,1661830741,none found
244,vicksburg,32.3526,-90.8779,80.28,97,0,0.00,US,1661830657,Hampton Inn & Suites Vicksburg


In [55]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Long"]]

In [56]:
# Add marker layer ontop of heat map
locations2=cities_df[['Lat','Long']]
humidity=cities_df['Humidity']
fig = gmaps.figure()
fig.add_layer(gmaps.heatmap_layer(locations2, weights=humidity,max_intensity=100))
markers=gmaps.marker_layer(hotel_df[['Lat','Long']], info_box_content=hotel_info)

fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))